In [1]:
import importlib as imp
import sys
import numpy as np
sys.path = list(set(sys.path + ['../../']))

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="6"
import tensorflow as tf

In [3]:
from sklearn import datasets
dataset_iris = datasets.load_iris(return_X_y=True)

In [4]:
def generate_sample_dataset_and_feature_dict(dataset_iris):
    """
        Make dataset from iris, assuming a user likes class 0 but dislikes class 1 and class 2
    """
    npd = np.zeros(len(dataset_iris[0]), dtype=[
        ('user_id', np.int32), ('item_id', np.int32), ('label', np.int32)])
    fea_dict = dict()
    for idx in range(len(dataset_iris[0])):
        if dataset_iris[1][idx] == 0:
            npd[idx] = (0, idx, 1)
        else:
            npd[idx] = (0, idx, -1)
        fea_dict[idx] = dataset_iris[0][idx]
    return npd, fea_dict

data_all, fea_dict = generate_sample_dataset_and_feature_dict(dataset_iris)

In [5]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(data_all)
print(len(data_train), len(data_test))

112 38


In [6]:
# create dataset instance from structured ndarray data
import recsys.dataset as dataset
dataset = imp.reload(dataset)
train_dataset = dataset.Dataset(data_train, total_users=1, total_items=len(fea_dict), implicit_negative=False, name='Train')
test_dataset = dataset.Dataset(data_test, total_users=1, total_items=len(fea_dict), implicit_negative=False, name='Test')

In [7]:
# create featurizer for mapping: (user, item) -> vector
import recsys.featurizer as featurizer
featurizer = imp.reload(featurizer)
class Featurizer(featurizer.FeaturizerBase):
    """
        Custom featurizer
    """
    def __init__(self, fea_dict):
        self.fea_dict = fea_dict

    def feature_dim(self):
        """
            Dim of X
        """
        return 4
    
    def featurize(self, user_id, item_id):
        if user_id != 0:
            raise "error"
        return self.fea_dict[item_id]

featurizer_iris = Featurizer(fea_dict)

In [8]:
# create data sampler for training and testing
import recsys.samplers.ranknet_sampler as ranknet_sampler
ranknet_sampler = imp.reload(ranknet_sampler)
train_sampler = ranknet_sampler.create_training_sampler(
    dataset=train_dataset, featurizer=featurizer_iris, max_pos_neg_per_user=5, batch_size=10, num_process=1, seed=100)
test_sampler = ranknet_sampler.create_evaluation_sampler(
    dataset=test_dataset, featurizer=featurizer_iris, max_pos_neg_per_user=5, seed=10)

In [9]:
# create evaluation metric
import recsys.metrics.auc as auc
auc = imp.reload(auc)
auc_evaluator = auc.AUC()

In [10]:
# create model
import recsys.recommenders.factorization_machine_recommender as factorization_machine_recommender
factorization_machine_recommender = imp.reload(factorization_machine_recommender)
model = factorization_machine_recommender.FactorizationMachineRecommender(feature_dim=featurizer_iris.feature_dim(),
    factor_dim=5, init_model_dir=None, save_model_dir='FMRec', l2_reg=None, train=True, serve=True)

In [11]:
# create model trainer with model
import recsys.model_trainer as model_trainer
model_trainer = imp.reload(model_trainer)
trainer = model_trainer.ModelTrainer(model=model)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [12]:
import datetime
print(datetime.datetime.now())

2019-07-26 17:07:58.488735


In [13]:
# train
trainer.train(total_iter=100, 
                    eval_iter=1,
                    save_iter=5,
                    train_sampler=train_sampler,
                    eval_samplers=[test_sampler], 
                    evaluators=[auc_evaluator])

[Training starts, total_iter: 100, eval_iter: 1, save_iter: 5]
[iter 1] loss: 0.006977
..(dataset: Test) evaluation
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 2] loss: 0.008076
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 3] loss: 0.016040
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 4] loss: 0.022005
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 5] Model saved.
[iter 5] loss: 0.026000
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 6] loss: 0.005454
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0


INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 47] loss: 0.006965
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 48] loss: 0.006015
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 49] loss: 0.001924
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
Instructions for updating:
Use standard file APIs to delete files with this prefix.
[iter 50] Model saved.
[iter 50] loss: 0.001731
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 51] loss: 0.013956
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 52] loss: 0.011319
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt


INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 93] loss: 0.004176
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 94] loss: 0.007070
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 95] Model saved.
[iter 95] loss: 0.008438
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 96] loss: 0.000892
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 97] loss: 0.000765
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 98] loss: 0.000658
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 99] loss: 0.002263
..(dataset: Test) evaluation
INFO

In [14]:
import datetime
print(datetime.datetime.now())

2019-07-26 17:08:13.365991


In [15]:
# Export Protobuf model for online serving
!rm -rf ./pbModel
model.export(export_model_dir="pbModel", as_text=False)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: pbModel/saved_model.pb


In [16]:
# inspect internal op tensor value of training
b = train_sampler.next_batch()
print("data:", b)
print("dy, dy_tilde:", model.train_inspect_ports(batch_data=b, ports=model.traingraph.get_outputs()))
print("loss:", model.train_inspect_ports(batch_data=b, ports=model.traingraph.get_losses()))

data: [([4.6, 3.6, 1. , 0.2], [5.5, 2.6, 4.4, 1.2], 1)
 ([5.5, 2.6, 4.4, 1.2], [4.6, 3.6, 1. , 0.2], 0)
 ([4.6, 3.6, 1. , 0.2], [6.5, 3. , 5.8, 2.2], 1)
 ([6.5, 3. , 5.8, 2.2], [4.6, 3.6, 1. , 0.2], 0)
 ([4.6, 3.6, 1. , 0.2], [6.4, 3.2, 5.3, 2.3], 1)
 ([6.4, 3.2, 5.3, 2.3], [4.6, 3.6, 1. , 0.2], 0)
 ([4.6, 3.6, 1. , 0.2], [5.6, 3. , 4.1, 1.3], 1)
 ([5.6, 3. , 4.1, 1.3], [4.6, 3.6, 1. , 0.2], 0)
 ([4.6, 3.6, 1. , 0.2], [6.9, 3.1, 4.9, 1.5], 1)
 ([6.9, 3.1, 4.9, 1.5], [4.6, 3.6, 1. , 0.2], 0)
 ([5.5, 4.2, 1.4, 0.2], [5.5, 2.6, 4.4, 1.2], 1)
 ([5.5, 2.6, 4.4, 1.2], [5.5, 4.2, 1.4, 0.2], 0)
 ([5.5, 4.2, 1.4, 0.2], [6.5, 3. , 5.8, 2.2], 1)
 ([6.5, 3. , 5.8, 2.2], [5.5, 4.2, 1.4, 0.2], 0)
 ([5.5, 4.2, 1.4, 0.2], [6.4, 3.2, 5.3, 2.3], 1)
 ([6.4, 3.2, 5.3, 2.3], [5.5, 4.2, 1.4, 0.2], 0)
 ([5.5, 4.2, 1.4, 0.2], [5.6, 3. , 4.1, 1.3], 1)
 ([5.6, 3. , 4.1, 1.3], [5.5, 4.2, 1.4, 0.2], 0)
 ([5.5, 4.2, 1.4, 0.2], [6.9, 3.1, 4.9, 1.5], 1)
 ([6.9, 3.1, 4.9, 1.5], [5.5, 4.2, 1.4, 0.2], 0)]
dy, dy_tilde:

In [17]:
# inspect internal op tensor value of testing
b = test_sampler.next_batch()
print("data:", b)
print("score:", model.serve_inspect_ports(batch_data=b[1], ports=model.servegraph.get_outputs()))

data: ([1, 1, 1, 1, 1, -1, -1, -1, -1, -1], array([([5. , 3.3, 1.4, 0.2],), ([4.4, 2.9, 1.4, 0.2],),
       ([5. , 3.4, 1.5, 0.2],), ([5.1, 3.8, 1.5, 0.3],),
       ([5.7, 3.8, 1.7, 0.3],), ([5.6, 3. , 4.5, 1.5],),
       ([6.9, 3.1, 5.4, 2.1],), ([6. , 3. , 4.8, 1.8],),
       ([6.3, 2.5, 4.9, 1.5],), ([5.7, 2.6, 3.5, 1. ],)],
      dtype=[('x', '<f4', (4,))]))
score: [array([ -3.4009924,  -3.2526047,  -3.4548223,  -3.3709354,  -4.0731387,
        -9.812959 , -13.034101 , -10.979875 , -11.310882 ,  -8.254461 ],
      dtype=float32)]


In [18]:
# inspect internal graph weights
def inspect_weights(checkpoint_model_dir):
    reader = tf.train.NewCheckpointReader(checkpoint_model_dir)
    variables = reader.get_variable_to_shape_map()
    for var, shape in variables.items():
        trained_weight = reader.get_tensor(var)
        print("VarName: {}".format(var))
        print("VarShape: {}".format(shape))
        print("VarValue: {}".format(trained_weight))
inspect_weights("./FMRec/model.ckpt")

VarName: optimizergraph/interactiongraph/LinearComponent/bias__0/Adam_1
VarShape: [1]
VarValue: [0.]
VarName: optimizergraph/interactiongraph/LinearComponent/bias__0/Adam
VarShape: [1]
VarValue: [0.]
VarName: optimizergraph/interactiongraph/LinearComponent/FC__0/Adam
VarShape: [4, 1]
VarValue: [[ 0.00118192]
 [-0.00236408]
 [ 0.00635234]
 [ 0.00241977]]
VarName: optimizergraph/interactiongraph/InteractiveComponent/interaction_factors/Adam
VarShape: [4, 5]
VarValue: [[ 0.00576144 -0.00582554 -0.00569561 -0.00589271  0.00584169]
 [ 0.00265497 -0.00266579 -0.00261007 -0.00268984  0.00266422]
 [-0.00367613  0.00357178  0.00364015  0.00362078 -0.00355306]
 [-0.00071166  0.00067765  0.00066842  0.00068366 -0.00066427]]
VarName: optimizergraph/beta2_power
VarShape: []
VarValue: 0.903888463973999
VarName: optimizergraph/beta1_power
VarShape: []
VarValue: 2.3905195121187717e-05
VarName: interactiongraph/LinearComponent/FC__0
VarShape: [4, 1]
VarValue: [[-0.82005537]
 [ 0.57761383]
 [-0.81760865